# Lesson 8: Advanced Queries with Subqueries

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-7  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Write subqueries in WHERE clauses
- Use subqueries in SELECT statements
- Work with IN, NOT IN, EXISTS, and NOT EXISTS
- Combine subqueries with JOINs
- Solve complex data retrieval problems
- Understand when to use subqueries vs JOINs


## 📚 What are Subqueries?

A **subquery** (or nested query) is a query inside another query. They allow you to:
- Use the results of one query in another
- Break complex problems into simpler steps
- Write more readable code
- Solve problems that would be difficult with just JOINs

### Real-World Example:

**Question:** "Find all characters from the same planet as Luke Skywalker"

**Without Subquery (2 steps):**
1. Query: What planet is Luke from? → Answer: Tatooine
2. Query: Who else is from Tatooine?

**With Subquery (1 step):**
```sql
SELECT name FROM characters 
WHERE homeworld_id = (SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker');
```

### Think of it Like:

Subqueries are like asking a question inside a question:
- "Show me students taller than **[the average height]**"
- The bracketed part is calculated first (subquery)
- Then used in the main query


## 🛠️ Setup: Connect to Database

**Run the next 3 cells:**


In [ ]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [ ]:
# Connect to the Star Wars database stored in database/
%sql sqlite:///database/starwars.db

## 🔍 Part 1: Subqueries in WHERE Clause

### Basic Subquery Syntax

```sql
SELECT columns
FROM table_name
WHERE column OPERATOR (SELECT column FROM table WHERE condition);
```

### Query 1: Find Luke's Homeworld First

**Step 1:** See what the subquery returns:


In [ ]:
%%sql
-- Query 1: What planet is Luke from?
SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker';


### Query 2: Use That in a Subquery

Now find all characters from the same planet:


In [ ]:
%%sql
-- Query 2: Find all characters from Luke's homeworld
SELECT name, species
FROM characters
WHERE homeworld_id = (SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker');


**Explanation:**

1. Subquery executes first: `(SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker')`
2. Returns a single value (e.g., 1)
3. Main query becomes: `WHERE homeworld_id = 1`
4. Finds all matches

**Key:** Subquery with `=` must return exactly ONE value.


### Query 3: Subquery with Comparison Operators

Find characters taller than average:


In [ ]:
%%sql
-- Query 3: Find characters taller than average
SELECT name, height
FROM characters
WHERE height > (SELECT AVG(height) FROM characters)
ORDER BY height DESC;


**Breakdown:**

1. Subquery: `(SELECT AVG(height) FROM characters)` → calculates average
2. Main query: Finds everyone taller than that average

**Alternative without subquery:** Would require calculating average separately, storing it, then querying.


### Query 4: Subquery with IN Operator

**IN** allows subqueries that return multiple values:


In [ ]:
%%sql
-- Query 4: Find characters from desert planets
SELECT name, species
FROM characters
WHERE homeworld_id IN (
    SELECT id 
    FROM planets 
    WHERE terrain LIKE '%desert%'
);


**Explanation:**

1. Subquery finds all planet IDs with desert terrain (returns multiple IDs)
2. `IN` checks if homeworld_id matches ANY of those IDs
3. Returns characters from any desert planet

**Why IN?** Because the subquery returns multiple rows (multiple desert planets).


### Query 5: Subquery with NOT IN

**NOT IN** excludes values:


In [ ]:
%%sql
-- Query 5: Find characters NOT from Tatooine or Alderaan
SELECT name, species
FROM characters
WHERE homeworld_id NOT IN (
    SELECT id 
    FROM planets 
    WHERE name IN ('Tatooine', 'Alderaan')
);


### Query 6: Nested Subqueries

Subqueries can be nested inside other subqueries:


In [ ]:
%%sql
-- Query 6: Characters from above-average population planets
SELECT c.name, p.name AS homeworld, p.population
FROM characters c
JOIN planets p ON c.homeworld_id = p.id
WHERE p.population > (
    SELECT AVG(population) 
    FROM planets 
    WHERE population IS NOT NULL
)
ORDER BY p.population DESC;


**This combines:**
- JOIN (to get planet names)
- Subquery (to calculate average population)
- WHERE (to filter)


## ✅ Part 2: EXISTS and NOT EXISTS

**EXISTS** checks if a subquery returns any rows. It's often more efficient than IN for large datasets.

### EXISTS vs IN

| Feature | EXISTS | IN |
|---------|--------|-----|
| Returns | TRUE/FALSE | Values to match |
| Performance | Usually faster | Can be slower |
| Readability | Clear intent | Also clear |
| Best for | Existence checks | Value matching |

### EXISTS Syntax

```sql
SELECT columns
FROM table1
WHERE EXISTS (SELECT 1 FROM table2 WHERE condition);
```

**Note:** `SELECT 1` is convention (just checks for existence, doesn't return actual data)

### Query 7: Using EXISTS


In [ ]:
%%sql
-- Query 7: Find characters who pilot at least one vehicle
SELECT name, species
FROM characters c
WHERE EXISTS (
    SELECT 1 
    FROM character_vehicles cv 
    WHERE cv.character_id = c.id
);


**Explanation:**

For each character:
1. Check if there's ANY row in character_vehicles with their ID
2. If yes (EXISTS returns TRUE), include them
3. If no, skip them

**Correlated Subquery:** The subquery references the outer query's table (`c.id`).


### Query 8: Using NOT EXISTS

Find characters who DON'T pilot any vehicles:


In [ ]:
%%sql
-- Query 8: Find characters who DON'T pilot any vehicles
SELECT name, species
FROM characters c
WHERE NOT EXISTS (
    SELECT 1 
    FROM character_vehicles cv 
    WHERE cv.character_id = c.id
);


**Alternative with LEFT JOIN:**

```sql
-- Same result using LEFT JOIN
SELECT c.name, c.species
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
WHERE cv.character_id IS NULL;
```

**NOT EXISTS is often:**
- More readable (clearer intent)
- Better performing
- Standard SQL approach


### Query 9: EXISTS with Correlated Subquery

Find planets that have at least one human:


In [ ]:
%%sql
-- Query 9: Find planets with at least one human
SELECT p.name AS planet_name, p.climate
FROM planets p
WHERE EXISTS (
    SELECT 1 
    FROM characters c 
    WHERE c.homeworld_id = p.id 
    AND c.species = 'Human'
);


**Correlated subquery explained:**

For each planet in the outer query:
1. Check if there's a character with `homeworld_id = p.id` AND `species = 'Human'`
2. The subquery "sees" the outer query's `p.id`
3. This correlation happens row-by-row


### Query 10: Complex EXISTS Example

Find characters who pilot the same type of vehicle as Luke:


In [ ]:
%%sql
-- Query 10: Characters who pilot same vehicle class as Luke
SELECT DISTINCT c.name
FROM characters c
WHERE EXISTS (
    SELECT 1
    FROM character_vehicles cv1
    JOIN vehicles v1 ON cv1.vehicle_id = v1.id
    WHERE cv1.character_id = c.id
    AND v1.vehicle_class IN (
        SELECT v2.vehicle_class
        FROM character_vehicles cv2
        JOIN vehicles v2 ON cv2.vehicle_id = v2.id
        JOIN characters c2 ON cv2.character_id = c2.id
        WHERE c2.name = 'Luke Skywalker'
    )
)
AND c.name != 'Luke Skywalker';


**This is complex!** Let's break it down:

1. **Outer query:** Loop through all characters
2. **EXISTS subquery:** For each character, check if they pilot...
3. **...a vehicle_class that's IN:**
4. **Nested subquery:** The vehicle classes Luke pilots
5. **AND c.name != 'Luke':** Don't include Luke himself

**Real-world translation:** "Show me everyone except Luke who flies the same type of vehicle as Luke.


## 📊 Part 3: Subqueries in SELECT

You can use subqueries in the SELECT list to add calculated columns.

### Query 11: Subquery as a Column


In [ ]:
%%sql
-- Query 11: Show each character with vehicle count
SELECT 
    c.name,
    c.species,
    (SELECT COUNT(*) 
     FROM character_vehicles cv 
     WHERE cv.character_id = c.id) AS vehicle_count
FROM characters c
ORDER BY vehicle_count DESC;


**Explanation:**

For each character:
1. Main query gets name and species
2. Subquery counts their vehicles
3. Result appears as a column

**Alternative:** Could use LEFT JOIN + GROUP BY, but subquery is sometimes clearer.


### Query 12: Multiple Subqueries in SELECT

Show how each character compares to average:


In [ ]:
%%sql
-- Query 12: Character height vs average
SELECT 
    name,
    height,
    (SELECT AVG(height) FROM characters WHERE height IS NOT NULL) AS avg_height,
    height - (SELECT AVG(height) FROM characters WHERE height IS NOT NULL) AS height_difference
FROM characters
WHERE height IS NOT NULL
ORDER BY height_difference DESC;


**Shows:** Who's tallest/shortest compared to average.

**Note:** The subquery runs for each row. For large datasets, consider calculating once and storing in a variable.


## 📊 Subquery vs JOIN: When to Use What?

### Use Subqueries When:

- ✅ You need a single aggregate value (AVG, MAX, MIN)
- ✅ Checking for existence (EXISTS/NOT EXISTS)
- ✅ Query logic is clearer with nesting
- ✅ You need calculated columns

### Use JOINs When:

- ✅ Combining columns from multiple tables
- ✅ Performance is critical
- ✅ You need multiple columns from related tables
- ✅ Working with very large datasets

### Example Comparison:


In [ ]:
%%sql
-- Subquery approach: Clear but runs subquery per row
SELECT 
    c.name,
    (SELECT COUNT(*) FROM character_vehicles cv WHERE cv.character_id = c.id) AS vehicles
FROM characters c;


In [ ]:
%%sql
-- JOIN approach: Usually faster
SELECT 
    c.name,
    COUNT(cv.vehicle_id) AS vehicles
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
GROUP BY c.name;


**Both produce same result!** Choose based on:
- Readability
- Performance needs
- Personal/team preference


## 🎓 Practice Exercises

### Exercise 1: Subquery with IN


In [ ]:
%%sql
-- Exercise 1: Characters from temperate climate planets
SELECT name, species
FROM characters
WHERE homeworld_id IN (
    SELECT id 
    FROM planets 
    WHERE climate = 'temperate'
);


### Exercise 2: NOT EXISTS


In [ ]:
%%sql
-- Exercise 2: Planets with no characters
SELECT name, climate
FROM planets p
WHERE NOT EXISTS (
    SELECT 1 
    FROM characters c 
    WHERE c.homeworld_id = p.id
);


### Exercise 3: Subquery with Comparison


In [ ]:
%%sql
-- Exercise 3: Characters shorter than all droids
SELECT name, species, height
FROM characters
WHERE height < (
    SELECT MIN(height) 
    FROM characters 
    WHERE species = 'Droid' AND height IS NOT NULL
)
AND height IS NOT NULL;


### Exercise 4: Complex Nested Subquery


In [ ]:
%%sql
-- Exercise 4: Vehicles piloted more than average
SELECT v.name, COUNT(cv.character_id) AS pilot_count
FROM vehicles v
JOIN character_vehicles cv ON v.id = cv.vehicle_id
GROUP BY v.name
HAVING COUNT(cv.character_id) > (
    SELECT AVG(pilot_cnt)
    FROM (
        SELECT COUNT(character_id) AS pilot_cnt
        FROM character_vehicles
        GROUP BY vehicle_id
    )
);


## 🐛 Common Errors & Troubleshooting

### Error: "Subquery returns more than 1 row"

**Problem:** Using `=` with a subquery that returns multiple values.

**Wrong:**
```sql
WHERE homeworld_id = (SELECT id FROM planets WHERE climate = 'temperate');
-- Error! Multiple temperate planets
```

**Correct:**
```sql
WHERE homeworld_id IN (SELECT id FROM planets WHERE climate = 'temperate');
-- IN handles multiple values
```

**Rule:** 
- Use `=` when subquery returns ONE value
- Use `IN` when subquery returns MULTIPLE values


### Slow Query Performance

**Problem:** Correlated subquery runs for every row.

**Less Efficient:**
```sql
SELECT name,
    (SELECT COUNT(*) FROM vehicles WHERE pilot_id = c.id)
FROM characters c;
-- Subquery runs once per character
```

**More Efficient:**
```sql
SELECT c.name, COUNT(v.id) AS vehicle_count
FROM characters c
LEFT JOIN vehicles v ON v.pilot_id = c.id
GROUP BY c.name;
-- JOIN runs once
```

**When to use each:**
- Subqueries: Clearer logic, small datasets
- JOINs: Better performance, large datasets


### Correlated Subquery Confusion

**Problem:** Forgetting to reference outer query.

**Wrong:**
```sql
WHERE EXISTS (SELECT 1 FROM planets WHERE population > 1000000);
-- Doesn't reference outer query! Always TRUE or FALSE for everyone
```

**Correct:**
```sql
WHERE EXISTS (
    SELECT 1 
    FROM planets p 
    WHERE c.homeworld_id = p.id 
    AND p.population > 1000000
);
-- Properly correlated with c.homeworld_id
```


### NULL Handling in Subqueries

**Problem:** NULLs cause unexpected results.

**Example:**
```sql
WHERE height > (SELECT AVG(height) FROM characters);
-- If any height is NULL, AVG might be unexpected
```

**Solution:**
```sql
WHERE height > (
    SELECT AVG(height) 
    FROM characters 
    WHERE height IS NOT NULL
);
-- Explicitly filter NULLs
```

**Rule:** Always consider NULLs in subqueries!


## 🎯 Challenge Problem

**Task:** Write a query using ONLY subqueries (no JOINs) that finds all characters who:

1. Are from the same planet as someone who pilots a starfighter
2. But do NOT pilot any starfighters themselves

**Requirements:**
- Use subqueries only
- Use IN or EXISTS
- Filter correctly

Try it yourself first!


### Challenge Solution


In [ ]:
%%sql
-- Challenge Solution
SELECT name, species
FROM characters
WHERE homeworld_id IN (
    -- Find planets of characters who pilot starfighters
    SELECT homeworld_id
    FROM characters
    WHERE id IN (
        SELECT character_id
        FROM character_vehicles
        WHERE vehicle_id IN (
            SELECT id
            FROM vehicles
            WHERE vehicle_class = 'Starfighter'
        )
    )
)
AND id NOT IN (
    -- Exclude characters who pilot starfighters
    SELECT character_id
    FROM character_vehicles
    WHERE vehicle_id IN (
        SELECT id
        FROM vehicles
        WHERE vehicle_class = 'Starfighter'
    )
);


**Breakdown:**

1. **Outermost WHERE:** Character must be from the right planet AND not pilot starfighters
2. **First IN subquery:** Find homeworld IDs of starfighter pilots
3. **Second NOT IN subquery:** Exclude anyone who pilots starfighters
4. **Nested subqueries:** Find starfighter IDs at multiple levels

**This demonstrates:** Complex logic possible with just subqueries!


## ✅ Checkpoint & Summary

### What You've Mastered

- ✅ Write subqueries in SELECT, FROM, and WHERE clauses
- ✅ Use correlated subqueries for row-by-row logic
- ✅ Combine EXISTS/NOT EXISTS with filtering
- ✅ Use CASE expressions for conditional output
- ✅ Leverage Common Table Expressions (CTEs) for readability
- ✅ Build advanced filtering strategies (ALL, ANY, IN)

### Advanced SQL Toolkit

| Feature | Purpose | Example |
|---------|---------|---------|
| Subquery | Nested query | `WHERE id IN (SELECT ...)` |
| Correlated Subquery | Refer outer query | `WHERE EXISTS (SELECT ... WHERE outer.id = inner.id)` |
| `EXISTS` | Check for matches | `WHERE EXISTS (...)` |
| `WITH` (CTE) | Reusable query block | `WITH stats AS (...) SELECT ...` |
| `CASE` | Conditional logic | `CASE WHEN ... THEN ... END` |
| Window Functions | Advanced analytics (preview) | `ROW_NUMBER() OVER (...)` |

## 🎉 Excellent Work!

You can now write complex nested queries and solve advanced data retrieval problems! In the next lesson, you'll integrate SQL with Python to build database-driven applications.

**Ready to continue?** Open `lesson9_database.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson8_advanced.ipynb database/starwars.db
git commit -m "Completed Lesson 8: Advanced queries with subqueries"
git push
```
